<p>&nbsp;</p>

### **Introduction**

Hello and welcome to my report for homework assignment 2+3 in IST 736 Text
Mining. For this analysis, I decided to use a new dataset instead of the dataset
from the first assignment. This is because in the first assignment, I had
already done most of the steps that are required for this assignment including
vectorization and implementing the multinomial naive bayes model.

I am excited about the new dataset because it is a dataset that I will be
collecting myself. The data that I will be collecting will consist of a sample
of NFL football related data. I will be collecting actual fantasy points and
projected fantasy points of starting offensive players in week 7, along with a
corpus of tweets related to those players. I will then be exploring the tweets
and applying machine learning.

The objective is to see if information from tweets can be used to predict whether
or not a player will outperform their projections for a given week.

<p>&nbsp;</p>

<font size = "3">

<p>&nbsp;</p>

### **Data Collection**

The data collection process was done in R because I was unable to figure out how
to do it in Python. I will be providing a copy of the R markdown file that goes
through the data collection process. The results from running the R code might be
different though because the tweets will be different. I will also be providing a
copy of the data being used in this report in a csv file.

I will summarize the data that I collected. I started off by retrieving the starting
rosters for week 7 NFL. I created a list of the players only from the QB, RB, WR, and
TE positions. I then collected each of these players projected fantasy points for
week 7. Unfortuantely I was not able to find a specific week 7 projection so I ended up
taking their season total projection and dividing it by the number of games. The oucome
variable is the variable that I am trying to predict in this report. 0 means the player
underperformed to their projection (less than 0.5 * projection), 1 means that they
were in line with thei projection (between 0.5 to 1.5 * projection) and 2 means that the
player outperformed their projection (greater than 1.5 * projection). Lastly, I scraped
100 tweets for each of the players using the player name as the key word in the search.

<p>&nbsp;</p>

<font size = "3">

<p>&nbsp;</p>

#### **Import Libraries**

<p>&nbsp;</p>

<font size = "3">

I will start off by loading some ofthe necessary packages. A short description of what I am using each package for is provided. For more information about the packages please look them up on the web.

In [2]:
# Import standard libraries

import numpy as np         # Working with numbers
import pandas as pd        # Working with dataframes
import matplotlib as plt   # Plotting
import tweepy              # Use the Twitter API
import re                  # Regular expression
import nltk                # Natural language tool kit

<p>&nbsp;</p>

#### **Import Data**

<p>&nbsp;</p>

<font size = "3">

I will load the data that I will be using for this analysis. As mentioned in the introduction, I have already collected this data in R and exported it to a csv format. The cleaning, stemming, and removing of stopwords has already been done. Here is a summary of the reduction of the vocabulary size from the corpus.

- original vocabulary = 146,224 words and 16,099 unique words
- all lowercase = 146,224 words and 14,185 unique words
- removed usernames = 142,606 words and 12,241 unique words
- removed http links = 141,203 words and 10,921 unique words
- removed punctuation tokens = 99,295 words and 9,368 unique words
- removed stopwords = 52.490 words and 8,988 unique words
- stemmed = 52,490 words and 7,632 unique words

In [6]:
# Import data

tweetsdf = pd.read_csv("tweets.csv")
tweets = tweetsdf["tweets"]
outcomes = tweetsdf["outcome"]

In [39]:
# Counts of the labels

outcomes.value_counts()

1    67
2    44
0    37
Name: outcome, dtype: int64

<p>&nbsp;</p>

#### **Vectorize**

<p>&nbsp;</p>

<font size = "3">

In the next section, I will be performing the necessary data preprocessing steps
to get the data in a form that can be used for machine learning. I will be doing
four different types of vectorization - boolean, count, relative, and tfidf. I am
not sure which one will be the best for the machine learning model so I would like
to try all of them.

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a function for vectorizing
def vectorize(tweets, labels, method):
  
  # Choose the vectorizer based on the method arg
  if method == "bool":
    vectorizer = TfidfVectorizer(binary = True)
  if method == "count":
    vectorizer = CountVectorizer()
  if method == "relative":
    vectorizer = TfidfVectorizer(use_idf = False, norm = "l1")
  # if method == "tfidf":
    # vectorizer == TfidfVectorizer()
  
  # Do the vectorization
  vectors = vectorizer.fit_transform(tweets)
  
  # Store the feature names
  features = vectorizer.get_feature_names()
  
  # Put it into a dataframe
  vectors_df = pd.DataFrame(vectors.toarray(), columns = features)
  
  # Add the outcomes
  vectors_df["outcomes"] = labels
  
  # Return the dataframe
  return vectors_df

In [17]:
# Perform the vectorization

tweets_bool = vectorize(tweets, outcomes, "bool")
tweets_count = vectorize(tweets, outcomes, "count")
tweets_rel = vectorize(tweets, outcomes, "relative")
# tweets_tfidf = vectorize(tweets, outcomes, "tfidf")

p>&nbsp;</p>

#### **Multinomial Naive Bayes Model**

<p>&nbsp;</p>

<font size = "3">

Finally, the data is preprocessed and ready to be used in a machine
learning model. The model of choice in this assignment will be the
multinomial naive bayes model. I will be using a 5 fold cross validation
to evaluate the performance of the model and I will do this on each
of the vectorization options.

In [18]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Define a function to train models

def trainer(train_predictors, train_labels, test_predictors, test_labels):
  bayes = MultinomialNB()
  bayes.fit(train_predictors, train_labels)
  print("Training Accuracy = ", bayes.score(train_predictors, train_labels))
  test_predictions = bayes.predict(test_predictors)
  print("Testing Accuracy = ", accuracy_score(test_labels, test_predictions))

In [22]:
from sklearn.model_selection import StratifiedKFold

# Define a function to run the model with 5 fold cv

def run_model(data, num_folds):

  split = StratifiedKFold(n_splits = num_folds, shuffle = True, random_state = 42)

  for train_index, test_index in split.split(data, data["outcomes"]):
    
    train_data = data.loc[train_index, :]
    test_data = data.loc[test_index, :]
    
    train_predictors = train_data.drop("outcomes", axis = 1)
    test_predictors = test_data.drop("outcomes", axis = 1)
    
    train_labels = train_data["outcomes"].copy()
    test_labels = test_data["outcomes"].copy()
    
    trainer(train_predictors, train_labels, test_predictors, test_labels)

In [24]:
# Run the model on the 3 different vectorization options

for dataframe in [tweets_bool, tweets_count, tweets_rel]:
    run_model(num_folds = 5, data = dataframe)
    print("\n")

Training Accuracy =  0.5
Testing Accuracy =  0.4666666666666667
Training Accuracy =  0.5
Testing Accuracy =  0.4666666666666667
Training Accuracy =  0.5
Testing Accuracy =  0.4666666666666667
Training Accuracy =  0.5042016806722689
Testing Accuracy =  0.4482758620689655
Training Accuracy =  0.5210084033613446
Testing Accuracy =  0.4482758620689655


Training Accuracy =  0.9491525423728814
Testing Accuracy =  0.43333333333333335
Training Accuracy =  0.9491525423728814
Testing Accuracy =  0.4
Training Accuracy =  0.9661016949152542
Testing Accuracy =  0.5333333333333333
Training Accuracy =  0.9495798319327731
Testing Accuracy =  0.4827586206896552
Training Accuracy =  0.9327731092436975
Testing Accuracy =  0.5172413793103449


Training Accuracy =  0.4576271186440678
Testing Accuracy =  0.43333333333333335
Training Accuracy =  0.4491525423728814
Testing Accuracy =  0.4666666666666667
Training Accuracy =  0.4491525423728814
Testing Accuracy =  0.4666666666666667
Training Accuracy =  0.4537

p>&nbsp;</p>

#### **Top Features**

<p>&nbsp;</p>

<font size = "3">

Based on the results of the naive bayes models, it looks like the
relative frequency vectorization produced the best results. There
were signs of overtraining in both the boolean and the count
vectorizations because the the training accuracy was much higher
than the test accuracy.

I will recreate the bayes model on the relative frequency vectorized
data now, and then I will evaluate the top features that the model
learned.

In [29]:
# Vectorize the data

vectorizer = TfidfVectorizer(use_idf = False, norm = "l1")
vectors = vectorizer.fit_transform(tweets)

In [30]:
# Separate the predictors and the labels

predictors = tweets_rel.drop("outcomes", axis = 1)
labels = tweets_rel["outcomes"].copy()

In [31]:
# Build the multinomial naive bayes model

bayes = MultinomialNB()
bayes.fit(predictors, labels)

MultinomialNB()

In [36]:
# Look at what it learned for above performance players

feature_ranks = sorted(zip(bayes.feature_log_prob_[2], vectorizer.get_feature_names()))
very_negative_features = feature_ranks[-10:]
print(very_negative_features)

[(-9.179608495084489, 'would'), (-9.145613669478841, 'get'), (-9.140887570746374, 'like'), (-9.135357730357507, 'yard'), (-9.106924202870706, 'game'), (-9.09214598726425, 'wr'), (-9.08986564529752, 'te'), (-9.06491979033236, 'play'), (-9.046515207487195, 'start'), (-8.911387845282169, 'week')]


In [37]:
# Look at what it learned for inline performance players

feature_ranks = sorted(zip(bayes.feature_log_prob_[1], vectorizer.get_feature_names()))
very_negative_features = feature_ranks[-10:]
print(very_negative_features)

[(-9.04746024845629, 'team'), (-9.037476467576205, 'qb'), (-9.0016298443785, 'get'), (-8.991921310219187, 'trade'), (-8.991539082984701, 'wr'), (-8.9669189850458, 'like'), (-8.928664278962128, 'game'), (-8.920223121855248, 'start'), (-8.912716913254654, 'play'), (-8.760562178194458, 'week')]


In [38]:
# Look at what it learned for below performance players

feature_ranks = sorted(zip(bayes.feature_log_prob_[0], vectorizer.get_feature_names()))
very_negative_features = feature_ranks[-10:]
print(very_negative_features)

[(-9.209476855875122, 'team'), (-9.206107867882043, 'te'), (-9.205950152646302, 'start'), (-9.202518313884752, 'like'), (-9.172651391730648, 'trade'), (-9.141575613858626, 'game'), (-9.123913153003343, 'play'), (-9.1207833888909, 'get'), (-9.067070069233766, 'wr'), (-9.055899064998462, 'week')]


<p>&nbsp;</p>

### **Conclusion**

The models did not perform as highly as I would have liked them to, so I am
unable to conclude in this analysis if the tweets were meaningful in determing
how a player performed in the game. All of the models seemed to be in about
the 40% - 50% accuracy range, while the majority vote baseline would be about
45% by just picking the outcome to be 1.

Looking at the top features, it looks like a lot of the same words were picked
up on by the model for the different classes which likely caused confusion and
noise in the model. It does not appear that the models have uncovered anything
insightful. It is possible that I could have done a better job with the data
cleaning, but due to time constraints I was unable to do so.

I believe that a major reason for this was the limitations that I faced during
the data collection process. First, the tweets that I collected were already
after the time the game had finished because I am unable to specifiy a date range
when using the twitter API due to restricted access. Ideally the tweets would
be for the week leading up the players game. Second, I was unable to get the players
true projection, I had to take an average of their season projection, which is not
ideal either because that projection changes every week.

In summary, unfortunately this project did not turn out the way that I hoped it would
for the amount of time that I spent on it. On a positive note, however, I got to
apply all of the concepts that I have learned about in the course, and I was able to get
the reticulate package in R working, even though I ended up moving back over to vs code
to finish the analysis. Despite the failure, with all else considered, this was
a good learning experience for me.

<p>&nbsp;</p>

<font size = "3">

<p>&nbsp;</p>

#### **References**

<p>&nbsp;</p>

<font size = "3">

https://stmorse.github.io/journal/pfr-scrape-python.html  
https://www.rstudio.com/resources/webinars/r-python-in-rstudio-1-2-with-reticulate/  
https://towardsdatascience.com/an-introduction-to-tweettokenizer-for-processing-tweets-9879389f8fe7